# How to create and add user made plugins to JDAViz

## Imports
Describe the libraries we're using here. If there's something unusual, explain what the library is, and why we need it.

- *astropy.units* to handle units
- *specutils* to work with Spectrum1D
- *jdaviz* to interact with the application

In [ ]:
from astropy import units as u
from glue.core.message import (DataCollectionAddMessage,
                               DataCollectionDeleteMessage)
from specutils import Spectrum1D

from jdaviz.core.registries import tools
from jdaviz.core.template_mixin import TemplateMixin
from jdaviz.utils import load_template
from jdaviz.app import Application

## Loading data

Load a 3D data cube into a cubeviz application.

In [ ]:
app = Application(configuration='cubeviz')
app.load_data("/Users/javerbukh/Documents/manga-7495-12704-LOGCUBE_fixed.fits")

app

## Connect button to toolbar

Create an *ipyvuetify* button and attach a method that fires when the button is clicked. Here, we are checking for the line flux at a particular wavelength. When the cell is run, you should see a button with a home icon in the center. We will then attach this button to jdaviz's toolbar in the following cell and if you look at the application above, you will see a new button added to the toolbar.

In [ ]:
import ipyvuetify as v

def line_flux(*args, **kwargs):
    input_wavelength = int(100)
    spec = app.session.data_collection[0].get_object(cls=Spectrum1D)

    # Takes the user input from the dialog (wavelength) and
    # finds the flux at that input

    print(spec.flux[input_wavelength])

plugin_button = v.Btn(children=[v.Icon(children=['home'])])
plugin_button.on_event('click', line_flux)

plugin_button

In [ ]:
toolbar = app.components.get('g-default-toolbar')
toolbar.tools = toolbar.tools + [plugin_button]

## Object-oriented plugin

We will now use an object-oriented approach for developing and attaching our plugin. This includes creating a class calling `LineFluxTool` and putting the code we want run when the button is clicked in the aptly named `clicked` method. Similarly to before, we will attach this class to the toolbar in the following cell. You should now see a button with a "compare arrows" icon to the right of the "home" icon in the application above.

In [ ]:
import ipyvuetify as v

class LineFluxTool(v.Btn):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        
        self.on_event('click', self.clicked)
        self.children = [v.Icon(children=['compare_arrows'])]
        
    def clicked(self, *args, **kwargs):
        input_wavelength = int(100)
        spec = app.session.data_collection[0].get_object(cls=Spectrum1D)

        # Takes the user input from the dialog (wavelength) and
        # finds the flux at that input

        print(spec.flux[input_wavelength])


In [ ]:
toolbar = app.components.get('g-default-toolbar')
toolbar.tools = toolbar.tools + [LineFluxTool()]

## Plugin with session on instantiation

We will now do the same as above except we will allow for the `app.session` instance to be passed in as a parameter, giving us access to the data within the session. After you run the following cells, you should see an "infinity" icon in the toolbar.

In [ ]:
import ipyvuetify as v

class LineFluxTool(v.Btn):
    def __init__(self, session, *args, **kwargs):
        super().__init__(*args, **kwargs)
        
        self.on_event('click', self.clicked)
        self.children = [v.Icon(children=['all_inclusive'])]
        self.session = session
        
    def clicked(self, *args, **kwargs):
        input_wavelength = int(100)
        
        # The difference is in this line
        spec = self.session.data_collection[0].get_object(cls=Spectrum1D)

        print(spec.flux[input_wavelength])


In [ ]:
toolbar = app.components.get('g-default-toolbar')

toolbar.tools = toolbar.tools + [LineFluxTool(app.session)]